### Fetch DB Queries and EDA
- Gets files from source
- Loads to DuckDB
- Transforms data to DWH

#### Tables
 - **users_raw** - raw users table from json
 - **dim_users** - transformed users table
 - **brands_raw** - raw brands table from json
 - **dim_brands** - transformed brands table
 - **receipts_raw** - raw receipts table from json
 - **receipts** - transformed receipts table
 - **receipt_line_item** - transformed reciept line item from receipts, many:1
 - **dim_date** -  role playing date dim

In [1]:
from fetch.ingestion.load_source_data import download_and_extract_gz,get_source_files
from fetch.ingestion.transform_data import unpack_receipts
from fetch.create_db import create_db
import duckdb

In [2]:
get_source_files()

Extracted gz file https://fetch-hiring.s3.amazonaws.com/analytics-engineer/ineeddata-data-modeling/receipts.json.gz to: ..\data
Extracted tar file https://fetch-hiring.s3.amazonaws.com/analytics-engineer/ineeddata-data-modeling/users.json.gz to: ..\data
Extracted gz file https://fetch-hiring.s3.amazonaws.com/analytics-engineer/ineeddata-data-modeling/brands.json.gz to: ..\data


In [3]:
create_db()

Created Dim Date
Create users_raw
created dim_users
created brands_raw
created dim_brands
created receipts_raw
created receipts
created receipt_line_item
Fetch DWH ready for analytics!!!


In [4]:
#test
sql="""
SELECT *
FROM receipts
limit 10
"""
duckdb.sql(sql)

┌──────────────────────────┬──────────────────────────┬──────────┬──────────────┬───────────────┬─────────────────────┬─────────────┬──────────────────────┬───────────────┬─────────────────────┐
│        receipt_id        │         user_id          │  status  │ date_scanned │ purchase_date │ points_awarded_date │ total_spent │ purchased_item_count │ points_earned │ bonus_points_earned │
│         varchar          │         varchar          │ varchar  │    int32     │     int32     │        int32        │    float    │        int32         │     float     │        float        │
├──────────────────────────┼──────────────────────────┼──────────┼──────────────┼───────────────┼─────────────────────┼─────────────┼──────────────────────┼───────────────┼─────────────────────┤
│ 5ff1e1eb0a720f0523000575 │ 5ff1e1eacfcf6c399c274ae6 │ FINISHED │     20210103 │      20210103 │            20210103 │        26.0 │                    5 │         500.0 │               500.0 │
│ 5ff1e1bb0a720f052300056

### SQL Questions

In [5]:
#What are the top 5 brands by receipts scanned for most recent month?
sql="""
with max_month as (
SELECT 
    max(start_of_month) as month
FROM
    receipt_line_item rl
    JOIN dim_date d on d.date_id = rl.date_scanned
    JOIN dim_brands b on b.brand_code = rl.brand_code
)
select
    b.brand_code
    ,mm.month
    ,count(*) as receipts_scanned
FROM
    receipt_line_item rl
    JOIN dim_date d on d.date_id = rl.date_scanned
    JOIN dim_brands b on b.brand_code = rl.brand_code
    JOIN max_month mm on d.start_of_month = mm.month
--WHERE start_of_month = (SELECT month from max_month)
GROUP BY all
ORDER BY receipts_scanned DESC
limit 5
"""
duckdb.sql(sql)

┌────────────┬────────────┬──────────────────┐
│ brand_code │   month    │ receipts_scanned │
│  varchar   │    date    │      int64       │
├────────────┼────────────┼──────────────────┤
│ VIVA       │ 2021-02-01 │                1 │
└────────────┴────────────┴──────────────────┘

In [6]:
#How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
sql="""
with max_month as (
SELECT 
    max(start_of_month) as month
FROM
    receipt_line_item rl
    JOIN dim_date d on d.date_id = rl.date_scanned
    JOIN dim_brands b on b.brand_code = rl.brand_code
)
,agg as (
select
    b.brand_code
    ,concat(d.year,'_',d.month) as year_month
    ,CASE
        when d.start_of_month = mm.month then 'current_month' else 'previous month'
    END relative_month
    ,count(*) as receipts_scanned
    
FROM
    receipt_line_item rl
    JOIN dim_date d on d.date_id = rl.date_scanned
    JOIN dim_brands b on b.brand_code = rl.brand_code
    JOIN max_month mm on d.start_of_month >= date_add(mm.month, - INTERVAL 1 MONTH)
GROUP BY all
)
SELECT *
,row_number() over(partition by year_month order by receipts_scanned desc nulls last) as _rank
FROM agg
qualify row_number() over(partition by year_month order by receipts_scanned desc nulls last) <= 5
order by year_month desc
"""
duckdb.sql(sql)


┌────────────┬────────────┬────────────────┬──────────────────┬───────┐
│ brand_code │ year_month │ relative_month │ receipts_scanned │ _rank │
│  varchar   │  varchar   │    varchar     │      int64       │ int64 │
├────────────┼────────────┼────────────────┼──────────────────┼───────┤
│ VIVA       │ 2021_2     │ current_month  │                1 │     1 │
│ KRAFT      │ 2021_1     │ previous month │               60 │     5 │
│ DORITOS    │ 2021_1     │ previous month │               77 │     4 │
│ KLEENEX    │ 2021_1     │ previous month │               88 │     2 │
│ KNORR      │ 2021_1     │ previous month │               79 │     3 │
│ PEPSI      │ 2021_1     │ previous month │               93 │     1 │
└────────────┴────────────┴────────────────┴──────────────────┴───────┘

In [7]:
#When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
#When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

sql="""
SELECT 
    status
    ,SUM(total_spent) as total_spend
    ,AVG(total_spent) as avg_spend
    ,SUM(purchased_Item_Count) as total_items_purchased
FROM
    receipts
WHERE
    status IN ('REJECTED','FINISHED','ACCEPTED') --can not find any accepted status
GROUP BY 1
"""
duckdb.sql(sql)

┌──────────┬────────────────────┬────────────────────┬───────────────────────┐
│  status  │    total_spend     │     avg_spend      │ total_items_purchased │
│ varchar  │       double       │       double       │        int128         │
├──────────┼────────────────────┼────────────────────┼───────────────────────┤
│ FINISHED │  41882.53011918068 │  80.85430524938356 │                  8184 │
│ REJECTED │ 1656.1499605178833 │ 23.326055781942017 │                   173 │
└──────────┴────────────────────┴────────────────────┴───────────────────────┘

### Data QA/EDA

 - Checekd column population
 - issues with user id uniqueness
 - issues with business key for brand dimension and how to join it to receipt line items
    - Orphaned records in line item with neither brand or barcode

In [8]:
import pandas as pd
import json

In [9]:
#checking % nulls for columns, looks like data is sparse for some
#espceially converning for columns I want to use as PK or business keys like brand code

files_dict = {
    "brands":"..\\data\\brands.json"
    ,"users":"..\\data\\users.json"
    ,"receipts":"..\\data\\receipts.json"
}

for k,v in files_dict.items():
    
    df = pd.read_json(v, lines=True)

    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                    'percent_missing': percent_missing})
    print('\n')
    print(k)
    print('....')
    print('\n')
    print(missing_value_df)



brands
....


               column_name  percent_missing
_id                    _id         0.000000
barcode            barcode         0.000000
category          category        13.281919
categoryCode  categoryCode        55.698372
cpg                    cpg         0.000000
name                  name         0.000000
topBrand          topBrand        52.442159
brandCode        brandCode        20.051414


users
....


               column_name  percent_missing
_id                    _id         0.000000
active              active         0.000000
createdDate    createdDate         0.000000
lastLogin        lastLogin        12.525253
role                  role         0.000000
signUpSource  signUpSource         9.696970
state                state        11.313131


receipts
....


                                     column_name  percent_missing
_id                                          _id         0.000000
bonusPointsEarned              bonusPointsEarned        51.385165
bonus

In [10]:
#Checking uniqueness, looks like we will need to de-duplicate some dimensions before using
sql="""
SELECT 
    _id as user_id
    ,count(*)
FROM
    users_raw
group by 1
having count(*) >1
order by 2 desc
"""
duckdb.sql(sql)

┌────────────────────────────────────┬──────────────┐
│              user_id               │ count_star() │
│       struct("$oid" varchar)       │    int64     │
├────────────────────────────────────┼──────────────┤
│ {'$oid': 5fc961c3b8cfca11a077dd33} │           20 │
│ {'$oid': 54943462e4b07e684157a532} │           20 │
│ {'$oid': 5fa41775898c7a11a6bcef3e} │           18 │
│ {'$oid': 59c124bae4b0299e55b0f330} │           18 │
│ {'$oid': 5ff5d15aeb7c7d12096d91a2} │           18 │
│ {'$oid': 600fb1ac73c60b12049027bb} │           16 │
│ {'$oid': 5ff1e194b6a9d73a3a9f1052} │           11 │
│ {'$oid': 600987d77d983a11f63cfa92} │            9 │
│ {'$oid': 600056a3f7e5b011fce897b0} │            8 │
│ {'$oid': 5a43c08fe4b014fd6b6a0612} │            8 │
│                 ·                  │            · │
│                 ·                  │            · │
│                 ·                  │            · │
│ {'$oid': 60074b49325c8a1794623876} │            2 │
│ {'$oid': 60005709bd4dff11d

In [11]:
#Checking what business key to use for brands, confused on barcode vs brand code, especially relating it to receipts
sql="""
SELECT 
    sum(case when brandcode is null then 1 else 0 end) as null_brand_code
    ,sum(case when barcode is null then 1 else 0 end) as null_barcode
    ,count(*) as total_records
FROM brands_raw
"""
duckdb.sql(sql)

┌─────────────────┬──────────────┬───────────────┐
│ null_brand_code │ null_barcode │ total_records │
│     int128      │    int128    │     int64     │
├─────────────────┼──────────────┼───────────────┤
│             234 │            0 │          1167 │
└─────────────────┴──────────────┴───────────────┘

In [12]:
#Seems inconsistent on how to join to receipt line item data, and lots or orphaned records
sql="""
   with t1 as (
        SELECT
            json_extract_string(_id, '$.$oid') as receipt_id
            ,unnest(rewardsReceiptItemList) as line_item
            ,*
        FROM
            receipts_raw
    )
    ,t2 as (
    SELECT 
        receipt_id
        ,json_extract_string(line_item, 'brandCode') as brand_code
        ,json_extract_string(line_item, 'barcode') as barcode
    FROM t1
    )
SELECT
    count(*) as total_line_items
    ,sum(case when brand_code is null then 1 else 0 end) as null_brand_code
    ,sum(case when barcode is null then 1 else 0 end) as null_barcode
    ,SUM(CASE WHEN barcode is null and brand_code is null then 1 else 0 end) orphaned_records
FROM t2
"""
duckdb.sql(sql)

┌──────────────────┬─────────────────┬──────────────┬──────────────────┐
│ total_line_items │ null_brand_code │ null_barcode │ orphaned_records │
│      int64       │     int128      │    int128    │      int128      │
├──────────────────┼─────────────────┼──────────────┼──────────────────┤
│             6941 │            4341 │         3851 │             2997 │
└──────────────────┴─────────────────┴──────────────┴──────────────────┘